In [1]:
import pandas as pd 
import numpy as np 
import scipy as sp 
des_size_1, des_size_2 = 3000, 5000 

In [2]:
data = pd.read_csv("train.csv", nrows = 1000)

In [3]:
data.head(10)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797
5,51e0962387f7,bbfad0b1ad0a,Татарстан,Чистополь,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Авто люлька,В хорошем состоянии,1300.0,9,2017-03-28,Private,eb6ad1231c59d3dc7e4020e724ffe8e4d302023ddcbb99...,796.0,0.80323
6,c4f260a2b48a,08f469d2e6f7,Нижегородская область,Нижний Новгород,Для дома и дачи,Ремонт и строительство,Сантехника и сауна,NaN,NaN,Водонагреватель 100 литров нержавейка плоский,Электро водонагреватель накопительный на 100 л...,11000.0,125,2017-03-23,Private,0330f6ac561f5db1fa8226dd5e7e127b5671d44d075a98...,2823.0,0.00000
7,6b71309d6a8a,fef86baa002c,Пермский край,Пермь,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Джинсы,26,Бойфренды colins,Бойфренды в хорошем состоянии.,500.0,61,2017-03-25,Private,9bab29a519e81c14f4582024adfebd4f11a4ac71d323a6...,567.0,0.80323
8,c5b969cb63a2,055825270190,Оренбургская область,Оренбург,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,> 50 (XXL),Платье,54 раз мер очень удобное,500.0,85,2017-03-17,Private,75ce06d1f939a31dfb2af8ac55f08fa998fa336d13ee05...,415.0,0.00000
9,b1570962e68c,f9e8f831d94c,Нижегородская область,Нижний Новгород,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,25,Полу ботиночки замш натур.Бамбини,По стельке 15.5см мерить приокский район. Цвет...,400.0,136,2017-03-22,Company,54fb8521135fda77a860bfd2fac6bf46867ab7c06796e3...,46.0,0.00000


In [4]:
city_list = data.city.unique()

In [5]:
des = data.description
des = des.fillna("0")
city = data.city.fillna("0")
u_type = data.user_type
cat_name = data.category_name.fillna("0")

from sklearn.feature_extraction.text import CountVectorizer 
vectorizer = CountVectorizer() 

corpus_1 = des.values
corpus_2 = city.values 
corpus_3 = u_type.values 
corpus_4 = cat_name.values 

X_1 = vectorizer.fit_transform(corpus_1) 
word_dic = vectorizer.get_feature_names()
X_2 = vectorizer.fit_transform(corpus_2)
city_dic = vectorizer.get_feature_names() 
X_3 = vectorizer.fit_transform(corpus_3)
u_type_dic = vectorizer.get_feature_names() 
X_4 = vectorizer.fit_transform(corpus_4) 
cat_dic = vectorizer.get_feature_names() 
print(X_1.toarray()[0])

[0 0 0 ..., 0 0 0]


In [6]:
corr = np.corrcoef(X_1.toarray(), data.deal_probability.values, rowvar = False)
words = pd.DataFrame(corr[:-1, -1], index=word_dic) # 単語と相関係数一覧
freq_words = words.sort_values(by=0)
freq_words = pd.concat((freq_words[:des_size_1], freq_words[-des_size_2:]))
description_vector = pd.DataFrame(X_1.toarray(), columns=word_dic) 
u_types = pd.DataFrame(X_3.toarray(), columns=u_type_dic)


cities_vec = pd.DataFrame(X_2.toarray(), columns=city_dic)
description_vector = pd.DataFrame(description_vector, columns=freq_words.index)
cat_vector = pd.DataFrame(X_4.toarray(), columns=cat_dic)
price = pd.DataFrame(np.log(data.price.fillna(1) + 1))

In [7]:
new_vector = pd.concat((cities_vec, description_vector, cat_vector), ignore_index = True, axis=1)
t = data.deal_probability.values

In [8]:
new_vector = new_vector.join(price)

In [9]:
vector_num = new_vector.values

In [26]:
np.save("train_X", vector_num) 
np.save("train_Y", t)

In [86]:
data.groupby('user_type')['deal_probability'].mean()

user_type
Company    0.179097
Private    0.217323
Shop       0.019706
Name: deal_probability, dtype: float64

In [87]:
data.groupby('user_type')['deal_probability'].std()

user_type
Company    0.316532
Private    0.330759
Shop       0.029539
Name: deal_probability, dtype: float64

In [83]:
data.category_name.unique().shape

(29,)

In [95]:
corr.shape

(9125, 9125)

In [101]:
words.sort_values(by=0).tail()

,0
коррекция,0.136207
работаю,0.141550
обрезка,0.143589
удаление,0.143589
загрузка,0.148903


False